<a href="https://colab.research.google.com/github/renanTrelarMadeiras/Receita_Federal_do_Brasil_-_Dados_Publicos_CNPJ/blob/master/ETL_coletar_dados_e_gravar_BD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. configurando ambiente

### INIT

In [ ]:
%pip install -r "/content/requeriment.txt"
# %pip install mysql-connector-python pandas pyarrow

In [ ]:
from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
import bs4 as bs
import ftplib
import gzip
import os
import pandas as pd
# import psycopg2
import re
import sys
import time
import requests
import urllib.request
import wget
import zipfile

import pyarrow.parquet as pq
from concurrent.futures import ThreadPoolExecutor

Ler arquivo de configuração de ambiente # https://dev.to/jakewitcher/using-env-files-for-environment-variables-in-python-applications-55a1

## DEFS

In [ ]:
# print('Especifique o local do seu arquivo de configuração ".env". Por exemplo: C:\...\Receita_Federal_do_Brasil_-_Dados_Publicos_CNPJ\code')
# C:\Aphonso_C\Git\Receita_Federal_do_Brasil_-_Dados_Publicos_CNPJ\code
# local_env = input()
local_env = "./"
# dotenv_path = os.path.join(local_env, '.env_template')
dotenv_path = os.path.join(local_env, 'env')
load_dotenv(dotenv_path=dotenv_path)

True

In [ ]:
def getEnv(env):
    return os.getenv(env)

In [ ]:
def check_diff(url, file_name):
    '''
    Verifica se o arquivo no servidor existe no disco e se ele tem o mesmo
    tamanho no servidor.
    '''
    if not os.path.isfile(file_name):
        return True # ainda nao foi baixado
    response = requests.head(url)
    new_size = int(response.headers.get('content-length', 0))
    old_size = os.path.getsize(file_name)
    if new_size != old_size:
        os.remove(file_name)
        return True # tamanho diferentes
    return False # arquivos sao iguais

In [ ]:
def makedirs(path):
    '''
    cria path caso seja necessario
    '''
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
def to_sql(dataframe, **kwargs):
    '''
    Quebra em pedacos a tarefa de inserir registros no banco
    '''
    size = 4096
    total = len(dataframe)
    name = kwargs.get('name')
    def chunker(df):
        return (df[i:i + size] for i in range(0, len(df), size))
    for i, df in enumerate(chunker(dataframe)):
        df.to_sql(**kwargs)
        index = i * size
        percent = (index * 100) / total
        progress = f'{name} {percent:.2f}% {index:0{len(str(total))}}/{total}'
        sys.stdout.write(f'\r{progress}')

In [ ]:
dados_rf = 'http://200.152.38.155/CNPJ/'

Read details from ".env" file:

In [ ]:
output_files = None
extracted_files = None
parquet_files = None
try:
    output_files = getEnv('OUTPUT_FILES_PATH')
    makedirs(output_files)
    extracted_files = getEnv('EXTRACTED_FILES_PATH')
    makedirs(extracted_files)

    parquet_files = getEnv('PATH_PARQUET')
    makedirs(parquet_files)

    print('Diretórios definidos: \n' +
          'output_files: ' + str(output_files)  + '\n' +
          'parquet_files: ' + str(parquet_files)  + '\n' +
          'extracted_files: ' + str(extracted_files)
          )
except:
    pass
    print('Erro na definição dos diretórios, verifique o arquivo ".env" ou o local informado do seu arquivo de configuração.')

Diretórios definidos: 
output_files: ./OUTPUT_FILES/
parquet_files: ./PATH_PARQUET
extracted_files: ./EXTRACTED_FILES/


In [ ]:
import pyarrow.parquet as pq

# def to_parquet(dataframe, file_name):
#     root = f"./{parquet_files}/"
#     dataframe.to_parquet(f'{root}{file_name}.parquet', index=False, compression='snappy')

def to_parquet(dataframe, file_name, append=True):
    root = f"./{parquet_files}/"
    file_path = f'{root}{file_name}.parquet'

    if append and os.path.exists(file_path):
        # Se o arquivo Parquet existir e a opção de append estiver ativada, acrescente os novos dados
        existing_data = pd.read_parquet(file_path)
        combined_data = pd.concat([existing_data, dataframe], ignore_index=True)
        combined_data.to_parquet(file_path, index=False, compression='snappy',engine="pyarrow")
        # print(f'Dados acrescentados a {file_path}')
    else:
        # Se o arquivo Parquet não existir ou a opção de append estiver desativada, crie um novo
        dataframe.to_parquet(file_path, index=False, compression='snappy',engine="pyarrow")
        # print(f'Novo arquivo Parquet criado em {file_path}')

# def to_parquet(dataframe, file_name):
#     root = f"./{parquet_files}/"
#     dataframe.to_parquet(f'{root}{file_name}.parquet', index=False, compression='snappy')

# def concat_parquet(file_name, file_name_2, final_file):
#     root = f"./{parquet_files}/"

#     existing_data = pd.read_parquet(root+file_name)
#     existing_data = pd.read_parquet(root+file_name_2)
#     combined_data = pd.concat([file_name, file_name_2], ignore_index=True)

#     combined_data.to_parquet(f"{root}+{final_file}.parquet", index=False, compression='brotli',engine="pyarrow")

def concat_parquet(conjunto):
    # import os

    root = f"./{parquet_files}/"
    # files = [name for name in os.listdir(root) if name.endswith('.parquet') and name.find(conjunto) ]
    files = [name for name in os.listdir(root) if name.find(conjunto) > -1 ]
    # files = [file_name,file_name_2]

    try:
      schema = pq.ParquetFile(root + files[0]).schema_arrow
      # print(root + files[0], files)
      # print(files[0])
      print(files)
      # print(schema)

      # with pq.ParquetWriter(root + conjunto + '_geral.parquet', schema=schema) as writer:
      #     for file in files:
      #         writer.write_table(pq.read_table(root + file, schema=schema))
    except :
      print(f"erro ao encontrar arquivos {conjunto}")
      pass


# 1. lendo urls

In [ ]:
raw_html = urllib.request.urlopen(dados_rf)
raw_html = raw_html.read()

Formatar página e converter em string

In [ ]:
page_items = bs.BeautifulSoup(raw_html, 'lxml')
html_str = str(page_items)

# 2. Coletar arquivos

## 2.1 Obter arquivos

In [ ]:
Files = []
text = '.zip'
for m in re.finditer(text, html_str):
    i_start = m.start()-40
    i_end = m.end()
    i_loc = html_str[i_start:i_end].find('href=')+6
    Files.append(html_str[i_start+i_loc:i_end])

Correcao do nome dos arquivos devido a mudanca na estrutura do HTML da pagina - 31/07/22 - Aphonso Rafael

In [ ]:
Files_clean = []
for i in range(len(Files)):
    if not Files[i].find('.zip">') > -1:
        Files_clean.append(Files[i])

In [ ]:
try:
    del Files
except:
    pass

In [ ]:
Files = Files_clean

In [ ]:
print('Arquivos que serão baixados:')
i_f = 0
for f in Files:
    i_f += 1
    print(str(i_f) + ' - ' + f)

Arquivos que serão baixados:
1 - Cnaes.zip
2 - Empresas0.zip
3 - Empresas1.zip
4 - Empresas2.zip
5 - Empresas3.zip
6 - Empresas4.zip
7 - Empresas5.zip
8 - Empresas6.zip
9 - Empresas7.zip
10 - Empresas8.zip
11 - Empresas9.zip
12 - Estabelecimentos0.zip
13 - Estabelecimentos1.zip
14 - Estabelecimentos2.zip
15 - Estabelecimentos3.zip
16 - Estabelecimentos4.zip
17 - Estabelecimentos5.zip
18 - Estabelecimentos6.zip
19 - Estabelecimentos7.zip
20 - Estabelecimentos8.zip
21 - Estabelecimentos9.zip
22 - Motivos.zip
23 - Municipios.zip
24 - Naturezas.zip
25 - Paises.zip
26 - Qualificacoes.zip
27 - Simples.zip
28 - Socios0.zip
29 - Socios1.zip
30 - Socios2.zip
31 - Socios3.zip
32 - Socios4.zip
33 - Socios5.zip
34 - Socios6.zip
35 - Socios7.zip
36 - Socios8.zip
37 - Socios9.zip


## 2.2 DOWNLOAD :

####################
 DOWNLOAD #################### <br>
Create this bar_progress method which is invoked automatically from wget:

In [ ]:
def bar_progress(current, total, width=80, arquivo=''):
  progress_message = "Downloading: %d%% [%d / %d] bytes - " % (current / total * 100, current, total)
  # Don't use print() as it will print in new line every time.
  sys.stdout.write(f"\r {arquivo}: {progress_message}")
  sys.stdout.flush()

#################### Download arquivos ####################

In [ ]:
# paralelizando download de arquivos
from concurrent.futures import ThreadPoolExecutor

def download_file(url, output_files):
    nome_arquivo = os.path.join(output_files, os.path.basename(url))
    if check_diff(url, nome_arquivo):
        wget.download(url, out=output_files, bar=bar_progress)

# Número de threads para usar no download paralelo
num_threads = 4 # Ajuste conforme necessário

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Use o executor para enviar tarefas de download em paralelo
    tarefas_download = [executor.submit(download_file, dados_rf + l, output_files) for l in Files]

    # Aguarde a conclusão de todas as tarefas
    for futuro in tarefas_download:
        futuro.result()


Download layout:

In [ ]:
Layout = 'https://www.gov.br/receitafederal/pt-br/assuntos/orientacao-tributaria/cadastros/consultas/arquivos/NOVOLAYOUTDOSDADOSABERTOSDOCNPJ.pdf'
print('Baixando layout:')
wget.download(Layout, out=output_files, bar=bar_progress)

Baixando layout:
 : Downloading: 100% [54000 / 54000] bytes - 

'./OUTPUT_FILES//NOVOLAYOUTDOSDADOSABERTOSDOCNPJ.pdf'

## 2.3 Extraindo Arquivos:

In [ ]:
# paralelizar extração de arquivos zip
def extract_zip(file_path, extracted_path):
    try:
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(extracted_path)
    except Exception as e:
        print(f"Erro ao extrair {file_path}: {e}")

# Número de threads para usar na extração paralela
num_threads = 5  # Ajuste conforme necessário

with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Use o executor para enviar tarefas de extração em paralelo
    tarefas_extracao = [executor.submit(extract_zip, os.path.join(output_files, l), extracted_files) for l in Files]

    # Aguarde a conclusão de todas as tarefas
    for futuro in tarefas_extracao:
        futuro.result()

KeyboardInterrupt: ignored

# 3. Ler e inserir dados

######################################################################################################################<br>
 LER E INSERIR DADOS #################################################################################################<br>
######################################################################################################################

## 3.0 carregar lista de dados locais

In [ ]:
insert_start = time.time()

Files:

In [ ]:
Items = [name for name in os.listdir(extracted_files) if name.endswith('')]

print(extracted_files)

./EXTRACTED_FILES/


Separar arquivos:

In [ ]:
arquivos_empresa = []
arquivos_estabelecimento = []
arquivos_socios = []
arquivos_simples = []
arquivos_cnae = []
arquivos_moti = []
arquivos_munic = []
arquivos_natju = []
arquivos_pais = []
arquivos_quals = []
for i in range(len(Items)):
    if Items[i].find('EMPRE') > -1:
        arquivos_empresa.append(Items[i])
    elif Items[i].find('ESTABELE') > -1:
        arquivos_estabelecimento.append(Items[i])
    elif Items[i].find('SOCIO') > -1:
        arquivos_socios.append(Items[i])
    elif Items[i].find('SIMPLES') > -1:
        arquivos_simples.append(Items[i])
    elif Items[i].find('CNAE') > -1:
        arquivos_cnae.append(Items[i])
    elif Items[i].find('MOTI') > -1:
        arquivos_moti.append(Items[i])
    elif Items[i].find('MUNIC') > -1:
        arquivos_munic.append(Items[i])
    elif Items[i].find('NATJU') > -1:
        arquivos_natju.append(Items[i])
    elif Items[i].find('PAIS') > -1:
        arquivos_pais.append(Items[i])
    elif Items[i].find('QUALS') > -1:
        arquivos_quals.append(Items[i])
    else:
        pass

Conectar no banco de dados:<br>
Dados da conexão com o BD

In [ ]:
user=getEnv('DB_USER')
passw=getEnv('DB_PASSWORD')
host=getEnv('DB_HOST')
port=getEnv('DB_PORT')
database=getEnv('DB_NAME')

Conectar:

In [ ]:
# conector para mysql
# engine = create_engine(f'mysql+mysqlconnector://{user}:{passw}@{host}:{port}/{database}')
# cur = conn.cursor()

# engine = create_engine('postgresql://'+user+':'+passw+'@'+host+':'+port+'/'+database)
# conn = psycopg2.connect('dbname='+database+' '+'user='+user+' '+'host='+host+' '+'port='+port+' '+'password='+passw)
# cur = conn.cursor()

## 3.1 Dados Empresas

In [ ]:
empresa_insert_start = time.time()
# print("""
# #######################
# ## Arquivos de EMPRESA:
# #######################
# """)


#######################
## Arquivos de EMPRESA:
#######################



Drop table antes do insert

In [ ]:

for e in range(0, len(arquivos_empresa)):
    print('Trabalhando no arquivo: '+arquivos_empresa[e]+' [...]')
    try:
        del empresa
    except:
        pass
    empresa = pd.DataFrame(columns=[0, 1, 2, 3, 4, 5, 6])
    empresa_dtypes = {0: 'object', 1: 'object', 2: 'object', 3: 'object', 4: 'object', 5: 'object', 6: 'object'}
    extracted_file_path = os.path.join(extracted_files, arquivos_empresa[e])
    empresa = pd.read_csv(filepath_or_buffer=extracted_file_path,
                          sep=';',
                          #nrows=100,
                          skiprows=0,
                          header=None,
                          dtype=empresa_dtypes,
                          encoding='latin-1'
    )

    # Tratamento do arquivo antes de inserir na base:
    empresa = empresa.reset_index()
    del empresa['index']

    # Renomear colunas
    empresa.columns = ['cnpj_basico', 'razao_social', 'natureza_juridica', 'qualificacao_responsavel', 'capital_social', 'porte_empresa', 'ente_federativo_responsavel']

    # Replace "," por "."
    empresa['capital_social'] = empresa['capital_social'].apply(lambda x: x.replace(',','.'))
    empresa['capital_social'] = empresa['capital_social'].astype(float)

    to_parquet(empresa,f'empresa_{arquivos_empresa[e]}')

    # to_sql(empresa, name='empresa', con=engine, if_exists='append', index=False)
    print('\t Arquivo ' + arquivos_empresa[e] + ' inserido com sucesso no banco de dados!')

Trabalhando no arquivo: K3241.K03200Y6.D31209.EMPRECSV [...]
Novo arquivo Parquet criado em ././PATH_PARQUET/empresa_K3241.K03200Y6.D31209.EMPRECSV.parquet
	 Arquivo K3241.K03200Y6.D31209.EMPRECSV inserido com sucesso no banco de dados!
Trabalhando no arquivo: K3241.K03200Y2.D31209.EMPRECSV [...]
Novo arquivo Parquet criado em ././PATH_PARQUET/empresa_K3241.K03200Y2.D31209.EMPRECSV.parquet
	 Arquivo K3241.K03200Y2.D31209.EMPRECSV inserido com sucesso no banco de dados!
Trabalhando no arquivo: K3241.K03200Y1.D31209.EMPRECSV [...]
Novo arquivo Parquet criado em ././PATH_PARQUET/empresa_K3241.K03200Y1.D31209.EMPRECSV.parquet
	 Arquivo K3241.K03200Y1.D31209.EMPRECSV inserido com sucesso no banco de dados!
Trabalhando no arquivo: K3241.K03200Y0.D31209.EMPRECSV [...]
Novo arquivo Parquet criado em ././PATH_PARQUET/empresa_K3241.K03200Y0.D31209.EMPRECSV.parquet
	 Arquivo K3241.K03200Y0.D31209.EMPRECSV inserido com sucesso no banco de dados!
Trabalhando no arquivo: K3241.K03200Y7.D31209.EMPREC

In [ ]:
# Items = [name for name in os.listdir(parquet_files) if name.endswith('.parquet') and name.find("EMPRE") ]
concat_parquet("EMPRE")

empresa_K3241.K03200Y6.D31209.EMPRECSV.parquet
['empresa_K3241.K03200Y6.D31209.EMPRECSV.parquet', 'empresa_K3241.K03200Y8.D31209.EMPRECSV.parquet', 'empresa_K3241.K03200Y3.D31209.EMPRECSV.parquet', 'empresa_K3241.K03200Y5.D31209.EMPRECSV.parquet', 'empresa_K3241.K03200Y9.D31209.EMPRECSV.parquet', 'empresa_K3241.K03200Y2.D31209.EMPRECSV.parquet', 'empresa_K3241.K03200Y7.D31209.EMPRECSV.parquet', 'empresa_K3241.K03200Y0.D31209.EMPRECSV.parquet', 'empresa_K3241.K03200Y4.D31209.EMPRECSV.parquet', 'empresa_K3241.K03200Y1.D31209.EMPRECSV.parquet']
cnpj_basico: string
razao_social: string
natureza_juridica: string
qualificacao_responsavel: string
capital_social: double
porte_empresa: string
ente_federativo_responsavel: string
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 1012


In [ ]:
try:
    del empresa
except:
    pass
print('Arquivos de empresa finalizados!')
empresa_insert_end = time.time()
empresa_Tempo_insert = round((empresa_insert_end - empresa_insert_start))
print('Tempo de execução do processo de empresa (em segundos): ' + str(empresa_Tempo_insert))

Arquivos de empresa finalizados!


NameError: ignored

## 3.2 Arquivos de estabelecimento:

In [ ]:
estabelecimento_insert_start = time.time()
print("""
###############################
## Arquivos de ESTABELECIMENTO:
###############################
""")

Drop table antes do insert

In [ ]:
# cur.execute('DROP TABLE IF EXISTS "estabelecimento";')
# conn.commit()

In [ ]:
for e in range(0, len(arquivos_estabelecimento)):
    print('Trabalhando no arquivo: '+arquivos_estabelecimento[e]+' [...]')
    try:
        del estabelecimento
    except:
        pass
    estabelecimento = pd.DataFrame(columns=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28])
    extracted_file_path = os.path.join(extracted_files, arquivos_estabelecimento[e])
    estabelecimento = pd.read_csv(filepath_or_buffer=extracted_file_path,
                          sep=';',
                          #nrows=100,
                          skiprows=0,
                          header=None,
                          dtype='object',
                          encoding='latin-1',
    )

    # Tratamento do arquivo antes de inserir na base:
    estabelecimento = estabelecimento.reset_index()
    del estabelecimento['index']

    # Renomear colunas
    estabelecimento.columns = ['cnpj_basico',
                               'cnpj_ordem',
                               'cnpj_dv',
                               'identificador_matriz_filial',
                               'nome_fantasia',
                               'situacao_cadastral',
                               'data_situacao_cadastral',
                               'motivo_situacao_cadastral',
                               'nome_cidade_exterior',
                               'pais',
                               'data_inicio_atividade',
                               'cnae_fiscal_principal',
                               'cnae_fiscal_secundaria',
                               'tipo_logradouro',
                               'logradouro',
                               'numero',
                               'complemento',
                               'bairro',
                               'cep',
                               'uf',
                               'municipio',
                               'ddd_1',
                               'telefone_1',
                               'ddd_2',
                               'telefone_2',
                               'ddd_fax',
                               'fax',
                               'correio_eletronico',
                               'situacao_especial',
                               'data_situacao_especial']

    # Gravar dados no banco:
    # estabelecimento
    # to_sql(estabelecimento, name='estabelecimento', con=engine, if_exists='append', index=False)
    to_sql(estabelecimento, name='estabelecimento', con=engine, if_exists='append', index=False)
    print('Arquivo ' + arquivos_estabelecimento[e] + ' inserido com sucesso no banco de dados!')

concat_parquet("ESTABELE")

IndentationError: ignored

In [ ]:
try:
    del estabelecimento
except:
    pass
print('Arquivos de estabelecimento finalizados!')
estabelecimento_insert_end = time.time()
estabelecimento_Tempo_insert = round((estabelecimento_insert_end - estabelecimento_insert_start))
print('Tempo de execução do processo de estabelecimento (em segundos): ' + str(estabelecimento_Tempo_insert))

## 3.3 Arquivos de socios:

In [ ]:
socios_insert_start = time.time()
print("""
######################
## Arquivos de SOCIOS:
######################
""")

Drop table antes do insert

In [ ]:
cur.execute('DROP TABLE IF EXISTS "socios";')
conn.commit()

In [ ]:
for e in range(0, len(arquivos_socios)):
    print('Trabalhando no arquivo: '+arquivos_socios[e]+' [...]')
    try:
        del socios
    except:
        pass
    extracted_file_path = os.path.join(extracted_files, arquivos_socios[e])
    socios = pd.DataFrame(columns=[1,2,3,4,5,6,7,8,9,10,11])
    socios = pd.read_csv(filepath_or_buffer=extracted_file_path,
                          sep=';',
                          #nrows=100,
                          skiprows=0,
                          header=None,
                          dtype='object',
                          encoding='latin-1',
    )

    # Tratamento do arquivo antes de inserir na base:
    socios = socios.reset_index()
    del socios['index']

    # Renomear colunas
    socios.columns = ['cnpj_basico',
                      'identificador_socio',
                      'nome_socio_razao_social',
                      'cpf_cnpj_socio',
                      'qualificacao_socio',
                      'data_entrada_sociedade',
                      'pais',
                      'representante_legal',
                      'nome_do_representante',
                      'qualificacao_representante_legal',
                      'faixa_etaria']

    # Gravar dados no banco:
    # socios
    to_sql(socios, name='socios', con=engine, if_exists='append', index=False)
    print('Arquivo ' + arquivos_socios[e] + ' inserido com sucesso no banco de dados!')

In [ ]:
try:
    del socios
except:
    pass
print('Arquivos de socios finalizados!')
socios_insert_end = time.time()
socios_Tempo_insert = round((socios_insert_end - socios_insert_start))
print('Tempo de execução do processo de sócios (em segundos): ' + str(socios_Tempo_insert))

## 3.4 Arquivos de simples:

In [ ]:
simples_insert_start = time.time()
print("""
################################
## Arquivos do SIMPLES NACIONAL:
################################
""")

Drop table antes do insert

In [ ]:
cur.execute('DROP TABLE IF EXISTS "simples";')
conn.commit()

In [ ]:
for e in range(0, len(arquivos_simples)):
    print('Trabalhando no arquivo: '+arquivos_simples[e]+' [...]')
    try:
        del simples
    except:
        pass

    # Verificar tamanho do arquivo:
    print('Lendo o arquivo ' + arquivos_simples[e]+' [...]')
    extracted_file_path = os.path.join(extracted_files, arquivos_simples[e])
    simples_lenght = sum(1 for line in open(extracted_file_path, "r"))
    print('Linhas no arquivo do Simples '+ arquivos_simples[e] +': '+str(simples_lenght))
    tamanho_das_partes = 1000000 # Registros por carga
    partes = round(simples_lenght / tamanho_das_partes)
    nrows = tamanho_das_partes
    skiprows = 0
    print('Este arquivo será dividido em ' + str(partes) + ' partes para inserção no banco de dados')
    for i in range(0, partes):
        print('Iniciando a parte ' + str(i+1) + ' [...]')
        simples = pd.DataFrame(columns=[1,2,3,4,5,6])
        simples = pd.read_csv(filepath_or_buffer=extracted_file_path,
                              sep=';',
                              nrows=nrows,
                              skiprows=skiprows,
                              header=None,
                              dtype='object',
                              encoding='latin-1',
        )

        # Tratamento do arquivo antes de inserir na base:
        simples = simples.reset_index()
        del simples['index']

        # Renomear colunas
        simples.columns = ['cnpj_basico',
                           'opcao_pelo_simples',
                           'data_opcao_simples',
                           'data_exclusao_simples',
                           'opcao_mei',
                           'data_opcao_mei',
                           'data_exclusao_mei']
        skiprows = skiprows+nrows

        # Gravar dados no banco:
        # simples
        to_sql(simples, name='simples', con=engine, if_exists='append', index=False)
        print('Arquivo ' + arquivos_simples[e] + ' inserido com sucesso no banco de dados! - Parte '+ str(i+1))
        try:
            del simples
        except:
            pass

In [ ]:
try:
    del simples
except:
    pass

In [ ]:
print('Arquivos do simples finalizados!')
simples_insert_end = time.time()
simples_Tempo_insert = round((simples_insert_end - simples_insert_start))
print('Tempo de execução do processo do Simples Nacional (em segundos): ' + str(simples_Tempo_insert))

## 3.5 Arquivos de cnae:

In [ ]:
cnae_insert_start = time.time()
print("""
######################
## Arquivos de cnae:
######################
""")

Drop table antes do insert

In [ ]:
cur.execute('DROP TABLE IF EXISTS "cnae";')
conn.commit()

In [ ]:
for e in range(0, len(arquivos_cnae)):
    print('Trabalhando no arquivo: '+arquivos_cnae[e]+' [...]')
    try:
        del cnae
    except:
        pass
    extracted_file_path = os.path.join(extracted_files, arquivos_cnae[e])
    cnae = pd.DataFrame(columns=[1,2])
    cnae = pd.read_csv(filepath_or_buffer=extracted_file_path, sep=';', skiprows=0, header=None, dtype='object', encoding='ANSI')

    # Tratamento do arquivo antes de inserir na base:
    cnae = cnae.reset_index()
    del cnae['index']

    # Renomear colunas
    cnae.columns = ['codigo', 'descricao']

    # Gravar dados no banco:
    # cnae
    to_sql(cnae, name='cnae', con=engine, if_exists='append', index=False)
    print('Arquivo ' + arquivos_cnae[e] + ' inserido com sucesso no banco de dados!')

In [ ]:
try:
    del cnae
except:
    pass
print('Arquivos de cnae finalizados!')
cnae_insert_end = time.time()
cnae_Tempo_insert = round((cnae_insert_end - cnae_insert_start))
print('Tempo de execução do processo de cnae (em segundos): ' + str(cnae_Tempo_insert))

## 3.6 Arquivos de moti:

In [ ]:
moti_insert_start = time.time()
print("""
#########################################
## Arquivos de motivos da situação atual:
#########################################
""")

Drop table antes do insert

In [ ]:
cur.execute('DROP TABLE IF EXISTS "moti";')
conn.commit()

In [ ]:
for e in range(0, len(arquivos_moti)):
    print('Trabalhando no arquivo: '+arquivos_moti[e]+' [...]')
    try:
        del moti
    except:
        pass
    extracted_file_path = os.path.join(extracted_files, arquivos_moti[e])
    moti = pd.DataFrame(columns=[1,2])
    moti = pd.read_csv(filepath_or_buffer=extracted_file_path, sep=';', skiprows=0, header=None, dtype='object', encoding='ANSI')

    # Tratamento do arquivo antes de inserir na base:
    moti = moti.reset_index()
    del moti['index']

    # Renomear colunas
    moti.columns = ['codigo', 'descricao']

    # Gravar dados no banco:
    # moti
    to_sql(moti, name='moti', con=engine, if_exists='append', index=False)
    print('Arquivo ' + arquivos_moti[e] + ' inserido com sucesso no banco de dados!')

In [ ]:
try:
    del moti
except:
    pass
print('Arquivos de moti finalizados!')
moti_insert_end = time.time()
moti_Tempo_insert = round((moti_insert_end - moti_insert_start))
print('Tempo de execução do processo de motivos da situação atual (em segundos): ' + str(moti_Tempo_insert))

## 3.7 Arquivos de munic:

In [ ]:
munic_insert_start = time.time()
print("""
##########################
## Arquivos de municípios:
##########################
""")

Drop table antes do insert

In [ ]:
cur.execute('DROP TABLE IF EXISTS "munic";')
conn.commit()

In [ ]:
for e in range(0, len(arquivos_munic)):
    print('Trabalhando no arquivo: '+arquivos_munic[e]+' [...]')
    try:
        del munic
    except:
        pass
    extracted_file_path = os.path.join(extracted_files, arquivos_munic[e])
    munic = pd.DataFrame(columns=[1,2])
    munic = pd.read_csv(filepath_or_buffer=extracted_file_path, sep=';', skiprows=0, header=None, dtype='object', encoding='ANSI')

    # Tratamento do arquivo antes de inserir na base:
    munic = munic.reset_index()
    del munic['index']

    # Renomear colunas
    munic.columns = ['codigo', 'descricao']

    # Gravar dados no banco:
    # munic
    to_sql(munic, name='munic', con=engine, if_exists='append', index=False)
    print('Arquivo ' + arquivos_munic[e] + ' inserido com sucesso no banco de dados!')

In [ ]:
try:
    del munic
except:
    pass
print('Arquivos de munic finalizados!')
munic_insert_end = time.time()
munic_Tempo_insert = round((munic_insert_end - munic_insert_start))
print('Tempo de execução do processo de municípios (em segundos): ' + str(munic_Tempo_insert))

## 3.8 Arquivos de natureza juridica:

In [ ]:
natju_insert_start = time.time()
print("""
#################################
## Arquivos de natureza jurídica:
#################################
""")

Drop table antes do insert

In [ ]:
cur.execute('DROP TABLE IF EXISTS "natju";')
conn.commit()

In [ ]:
for e in range(0, len(arquivos_natju)):
    print('Trabalhando no arquivo: '+arquivos_natju[e]+' [...]')
    try:
        del natju
    except:
        pass
    extracted_file_path = os.path.join(extracted_files, arquivos_natju[e])
    natju = pd.DataFrame(columns=[1,2])
    natju = pd.read_csv(filepath_or_buffer=extracted_file_path, sep=';', skiprows=0, header=None, dtype='object', encoding='ANSI')

    # Tratamento do arquivo antes de inserir na base:
    natju = natju.reset_index()
    del natju['index']

    # Renomear colunas
    natju.columns = ['codigo', 'descricao']

    # Gravar dados no banco:
    # natju
    to_sql(natju, name='natju', con=engine, if_exists='append', index=False)
    print('Arquivo ' + arquivos_natju[e] + ' inserido com sucesso no banco de dados!')

In [ ]:
try:
    del natju
except:
    pass
print('Arquivos de natju finalizados!')
natju_insert_end = time.time()
natju_Tempo_insert = round((natju_insert_end - natju_insert_start))
print('Tempo de execução do processo de natureza jurídica (em segundos): ' + str(natju_Tempo_insert))

## 3.9 Arquivos de pais:

In [ ]:
pais_insert_start = time.time()
print("""
######################
## Arquivos de país:
######################
""")

Drop table antes do insert

In [ ]:
cur.execute('DROP TABLE IF EXISTS "pais";')
conn.commit()

In [ ]:
for e in range(0, len(arquivos_pais)):
    print('Trabalhando no arquivo: '+arquivos_pais[e]+' [...]')
    try:
        del pais
    except:
        pass
    extracted_file_path = os.path.join(extracted_files, arquivos_pais[e])
    pais = pd.DataFrame(columns=[1,2])
    pais = pd.read_csv(filepath_or_buffer=extracted_file_path, sep=';', skiprows=0, header=None, dtype='object', encoding='ANSI')

    # Tratamento do arquivo antes de inserir na base:
    pais = pais.reset_index()
    del pais['index']

    # Renomear colunas
    pais.columns = ['codigo', 'descricao']

    # Gravar dados no banco:
    # pais
    to_sql(pais, name='pais', con=engine, if_exists='append', index=False)
    print('Arquivo ' + arquivos_pais[e] + ' inserido com sucesso no banco de dados!')

In [ ]:
try:
    del pais
except:
    pass
print('Arquivos de pais finalizados!')
pais_insert_end = time.time()
pais_Tempo_insert = round((pais_insert_end - pais_insert_start))
print('Tempo de execução do processo de país (em segundos): ' + str(pais_Tempo_insert))

## 3.10 Arquivos de qualificação de sócios:

In [ ]:
quals_insert_start = time.time()
print("""
######################################
## Arquivos de qualificação de sócios:
######################################
""")

Drop table antes do insert

In [ ]:
cur.execute('DROP TABLE IF EXISTS "quals";')
conn.commit()

In [ ]:
for e in range(0, len(arquivos_quals)):
    print('Trabalhando no arquivo: '+arquivos_quals[e]+' [...]')
    try:
        del quals
    except:
        pass
    extracted_file_path = os.path.join(extracted_files, arquivos_quals[e])
    quals = pd.DataFrame(columns=[1,2])
    quals = pd.read_csv(filepath_or_buffer=extracted_file_path, sep=';', skiprows=0, header=None, dtype='object', encoding='ANSI')

    # Tratamento do arquivo antes de inserir na base:
    quals = quals.reset_index()
    del quals['index']

    # Renomear colunas
    quals.columns = ['codigo', 'descricao']

    # Gravar dados no banco:
    # quals
    to_sql(quals, name='quals', con=engine, if_exists='append', index=False)
    print('Arquivo ' + arquivos_quals[e] + ' inserido com sucesso no banco de dados!')

In [ ]:
try:
    del quals
except:
    pass
print('Arquivos de quals finalizados!')
quals_insert_end = time.time()
quals_Tempo_insert = round((quals_insert_end - quals_insert_start))
print('Tempo de execução do processo de qualificação de sócios (em segundos): ' + str(quals_Tempo_insert))

%

In [ ]:
insert_end = time.time()
Tempo_insert = round((insert_end - insert_start))

# 4 Finalizando

In [ ]:
print("""
#############################################<br>
## Processo de carga dos arquivos finalizado!<br>
#############################################<br>
""")

In [ ]:
print('Tempo total de execução do processo de carga (em segundos): ' + str(Tempo_insert)) # Tempo de execução do processo (em segundos): 17.770 (4hrs e 57 min)

###############################<br>
Tamanho dos arquivos:<br>
empresa = 45.811.638<br>
estabelecimento = 48.421.619<br>
socios = 20.426.417<br>
simples = 27.893.923<br>
###############################

%<br>
Criar índices na base de dados:

In [ ]:
index_start = time.time()
print("""
#######################################
## Criar índices na base de dados [...]
#######################################
""")
cur.execute("""
create index empresa_cnpj on empresa(cnpj_basico);
commit;
create index estabelecimento_cnpj on estabelecimento(cnpj_basico);
commit;
create index socios_cnpj on socios(cnpj_basico);
commit;
create index simples_cnpj on simples(cnpj_basico);
commit;
""")
conn.commit()
print("""
############################################################
## Índices criados nas tabelas, para a coluna `cnpj_basico`:
   - empresa
   - estabelecimento
   - socios
   - simples
############################################################
""")
index_end = time.time()
index_time = round(index_end - index_start)
print('Tempo para criar os índices (em segundos): ' + str(index_time))

%

print(
Processo 100% finalizado! Você já pode usar seus dados no BD!<br>
 - Desenvolvido por: Aphonso Henrique do Amaral Rafael<br>
 - Contribua com esse projeto aqui: https://github.com/aphonsoar/Receita_Federal_do_Brasil_-_Dados_Publicos_CNPJ<br>
